# Helpful Docs
- [API Key GPT best practices](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety)
- [GPT OpenAI API Docs](https://platform.openai.com/docs/api-reference/introduction)
- [Swagger Bearer Auth](https://swagger.io/docs/specification/v3_0/authentication/bearer-authentication/)
- 

# Dependencies

### Mesa
`pip install mesa`

### Solara [For Visualization]:
https://solara.dev/documentation/getting_started/installing

1) `python -m venv solara-env`
2) `source ./solara-env/bin/activate`
3) `pip install solara`

### networkx

`pip install networkx`

### matplotlib
`pip install matplotlib`

### altair

`pip install altair`

# ChatGPT Request Dependencies:

### OpenAI
`pip install openai`



In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import mesa
import requests
import os
import sys
import random
from openai import OpenAI



: 

In [ ]:
# API key steup
key = os.environ["GPT_GAME_KEY"]

client = OpenAI(api_key=key)

# response = client.responses.create(
#     model="gpt-5-mini",
#     input="Pretend to be a League of Legends player. Write something you would say in a chat."
# )

#print(response.output_text)


In [ ]:
HIT_GPT = False

class AiModAgent(mesa.Agent):
    def __init__(self, model):
        # Pass the parameters to the parent class.
        super().__init__(model)

class PlayerAgent(mesa.Agent):
    def __init__(self, model):
        # Pass the parameters to the parent class.
        super().__init__(model)
        self.current_message = ""

    def chat(self):
        respond=random.binomialvariate(n=1,p=0.5)
        if(respond == 1):
            self.current_message=self.get_message()
            self.model.chat_log = self.model.chat_log + (f"\n player {self.unique_id}: {self.current_message}")
        # Decide if player responds
        # Decide what message(s?) to respond to
        # update the chat log here
        if(respond == 0):
            self.model.chat_log = " "

    def get_message(self):
        prompt = self.get_prompt()
        if(HIT_GPT==True):
            response = client.responses.create(
                model="gpt-5-mini",
                input=prompt
            )
            return response.output_text
        else:
            return "This is a chat message"
        

    def get_prompt(self):
        return f"Pretend to be player {self.unique_id} in a League of Legends game. {model.is_winning()}. Write one message you would say in the game chat. This is the current chat history: {model.chat_log}"
        
        

class GameModel(mesa.Model):
    
    def __init__(self,num_players,end_game_pts):
        super().__init__()
        self.chat_log = ""
        self.num_players = num_players
        self.ending_game_points = end_game_pts # the number of points a team needs to have to end the game
        self.datacollector = mesa.DataCollector(
            model_reporters={"Game state":self.is_winning},
            agent_reporters={"current_message": "current_message"}
        )
        # self.chat
        # self.personas
        agents=PlayerAgent.create_agents(self,n=self.num_players)
        self.is_game_done=False
        self.points = {
            "team":0,
            "opponent":0}
        
    
    def step(self):
        self.progress_game()
        self.agents.shuffle_do("chat")
        self.datacollector.collect(self)
        # dump/collect chat log in data collector
    
    def progress_game(self):
        coin_flip=random.binomialvariate(n=1,p=0.5) # Idea: abstract game into a cointoss game
        #update points based on the game outcome
        if(coin_flip==1):
            self.points["team"] = self.points["team"] + 1
        else:
            self.points["opponent"] = self.points["opponent"] + 1
            
        if((self.points["team"] == self.ending_game_points)or (self.points["opponent"] == self.ending_game_points)):
            self.is_game_done = True

    def is_winning(self):
        if(self.is_game_done == True):
            return "The game is done"
            
        if(self.points["team"]>self.points["opponent"]):
            return "Your team is winning"
        elif(self.points["team"]<self.points["opponent"]):
            return "Your team is losing"
        elif(self.points["team"]==self.points["opponent"]):
            return "The game is tied"
        



In [ ]:
model = GameModel(5,5)

while(model.is_game_done==False):
    model.step()

print(model.points)
game_df=model.datacollector.get_model_vars_dataframe()
agents_df=model.datacollector.get_agent_vars_dataframe()
print(game_df)
print(agents_df)
print(model.chat_log)

{'team': 1, 'opponent': 5}
             Game state
0  Your team is winning
1      The game is tied
2   Your team is losing
3   Your team is losing
4   Your team is losing
5      The game is done
                     current_message
Step AgentID                        
1    1        This is a chat message
     2                              
     3                              
     4        This is a chat message
     5        This is a chat message
2    1        This is a chat message
     2                              
     3                              
     4        This is a chat message
     5        This is a chat message
3    1        This is a chat message
     2        This is a chat message
     3        This is a chat message
     4        This is a chat message
     5        This is a chat message
4    1        This is a chat message
     2        This is a chat message
     3        This is a chat message
     4        This is a chat message
     5        This is a chat

In [ ]:
agents_df=agents_df.rename(columns={"AgentID":"PlayerID"})
agents_df.to_csv("test.csv")
game_df.to_csv("game_outcome.csv")